# Concatenation and merging

Merging data from different sources can be very daunting. Data can have diverse type, some rows or columns can be present only in one of the DaraFrames, etc. Historically, the problems of merging were addressed by databases, such as SQL. `Pandas` provides a lot of functionality in this domain.

In [17]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [18]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [19]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [20]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1,'\n')
print(df2)
pd.concat([df1, df2])

    A   B
1  A1  B1
2  A2  B2 

    A   B
3  A3  B3
4  A4  B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Pandas concatenation preserves indices, even if the result will have duplicate indices!

In [21]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [2, 3])
print(df1,'\n')
print(df2)
pd.concat([df1, df2])

    A   B
1  A1  B1
2  A2  B2 

    A   B
2  A2  B2
3  A3  B3


,A,B
1,A1,B1
2,A2,B2
2,A2,B2
3,A3,B3


In [22]:
pd.concat([df1, df2], keys=['df1', 'df2'])

A   B
df1 1  A1  B1
    2  A2  B2
df2 2  A2  B2
    3  A3  B3

In [23]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
pd.concat([df5, df6])

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


In [24]:
pd.concat([df5, df6], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


In [25]:
pd.concat([df5, df6], join_axes=[df5.columns])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


# `pd.merge`

If merging data sounds confusing to you at any point, it is because it is. There is a whole area of math called *relational algebra*, which creates the theoretical underpinnings of databases and how they work. We won't study any of that, not only it requires a course of its own, but it is also not very frequent for scientists to deal with the kind of data, which requires database to be dealt with efficiently.

Nevertheless, joining DataFrames can come in handy, and sometimes there joins can be quite tricky, so we will learn a bit of that. In general there are 3 types of joins: one-to-one, many-to-one, and many-to-many. Pandas function `merge()` provides an interface to do all of the, depending on the inputs. The first (one-to-one) referers to the simplest case, when you have 2 sources, and none of them have duplicate entries. In this case joining is usually easy, and basically reminds a concatenation:

In [7]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [8]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


One-to-many is when one of your `DataFrames` contains duplicate entries. `pd.merge` will understand that and try to fill in the values appropriately:

In [9]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

Many-to-many is the most confusing type of join, but it is nevertheless well defined mathematically. Consider the following, where we have a `DataFrame` showing one or more skills associated with a particular group. By performing a many-to-many join, we can recover the skills associated with any individual person. Note that some entries in both `df1` and `df5` had to be duplicated; also "R&D" group disappeared in the joined `DataFrame`, because it had no pairings withing `df1`.

In [13]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'R&D'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'science']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Here we have only scratch the surface of merging. If you ever need to do complicated joins or are just interested in learning more about joins with `pandas`, I highly recommend <a href="http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.07-Merge-and-Join.ipynb">Combining Datasets: Merge and Join</a> section on the "Python Data Science Handbook" by Jake VanderPlas and <a href="http://pandas.pydata.org/pandas-docs/stable/merging.html">Merge, join, and concatenate</a> section of the `Pandas` documentation.

# Aggregation and groupping

In [26]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

D:\Programs\Anaconda\envs\py36\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


(1035, 6)

In [28]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [32]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


<img src="http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/03.08-split-apply-combine.png"></img>
From "Aggregation and groupping chapter" of "Python Data Science Handbook" by Jake VanderPlas

In [33]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [34]:
df.groupby('key')

In [35]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


In [36]:
planets.groupby('method')

In [37]:
planets.groupby('method')['orbital_period']

In [38]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [81]:
for key, group in planets.groupby(['method'])['orbital_period']:
    print(key, group.mean())

Astrometry 631.18
Eclipse Timing Variations 4751.64444444
Imaging 118247.7375
Microlensing 3153.57142857
Orbital Brightness Modulation 0.709306583333
Pulsar Timing 7343.02120126
Pulsation Timing Variations 1170.0
Radial Velocity 823.354680017
Transit 21.1020726713
Transit Timing Variations 79.7835


In [86]:
for key, group in planets.groupby(['method'])['orbital_period']:
    print('{:35s} {}'.format(key, group.mean()))

Astrometry                          631.1800000000001
Eclipse Timing Variations           4751.644444444445
Imaging                             118247.7375
Microlensing                        3153.5714285714284
Orbital Brightness Modulation       0.7093065833333334
Pulsar Timing                       7343.021201258
Pulsation Timing Variations         1170.0
Radial Velocity                     823.3546800171247
Transit                             21.102072671259446
Transit Timing Variations           79.7835


In [83]:
planets.groupby('method').mean()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,1.000000,631.180000,NaN,17.875000,2011.500000
Eclipse Timing Variations,1.666667,4751.644444,5.125000,315.360000,2010.000000
Imaging,1.315789,118247.737500,NaN,67.715937,2009.131579
Microlensing,1.173913,3153.571429,NaN,4144.000000,2009.782609
Orbital Brightness Modulation,1.666667,0.709307,NaN,1180.000000,2011.666667
Pulsar Timing,2.200000,7343.021201,NaN,1200.000000,1998.400000
Pulsation Timing Variations,1.000000,1170.000000,NaN,NaN,2007.000000
Radial Velocity,1.721519,823.354680,2.630699,51.600208,2007.518987
Transit,1.954660,21.102073,1.470000,599.298080,2011.236776


In [84]:
planets.groupby(['method','number']).mean()

orbital_period      mass     distance  \
method                        number                                          
Astrometry                    1           631.180000       NaN    17.875000   
Eclipse Timing Variations     1          5821.166667  5.125000          NaN   
                              2          4216.883333       NaN   315.360000   
Imaging                       1        140621.606250       NaN    71.683929   
                              4         73500.000000       NaN    39.940000   
Microlensing                  1          3030.000000       NaN  4160.000000   
                              2          3462.500000       NaN  4080.000000   
Orbital Brightness Modulation 1             1.544929       NaN          NaN   
                              2             0.291496       NaN  1180.000000   
Pulsar Timing                 1         18262.545353       NaN  1200.000000   
                              3            63.338433       NaN          NaN   
Pulsation Timing Variations   1          1170.000000       NaN          NaN   
Radial Velocity               1           814.143794  3.323939    60.648243   
                              2           959.022946  2.229547    49.316190   
                              3           769.421724  0.916872    30.302308   
                              4           695.769616  0.986492     9.930000   
                              5          1045.993508  1.166750    12.530000   
                              6           213.752357  0.038556    19.673333   
Transit                       1            12.301023  1.470000   584.836460   
                              2            27.589652       NaN   924.142857   
                              3            22.187586       NaN   481.007143   
                              4            21.430775       NaN          NaN   
                              5            30.584464       NaN   320.333333   
                              6            40.513600       NaN   613.000000   
                              7           119.217898       NaN   780.000000   
Transit Timing Variations     2            79.783500       NaN  1104.333333   
                              3                  NaN       NaN          NaN   

                                             year  
method                        number               
Astrometry                    1       2011.500000  
Eclipse Timing Variations     1       2010.666667  
                              2       2009.666667  
Imaging                       1       2009.205882  
                              4       2008.500000  
Microlensing                  1       2009.736842  
                              2       2010.000000  
Orbital Brightness Modulation 1       2013.000000  
                              2       2011.000000  
Pulsar Timing                 1       2007.000000  
                              3       1992.666667  
Pulsation Timing Variations   1       2007.000000  
Radial Velocity               1       2007.082317  
                              2       2008.219697  
                              3       2008.886792  
                              4       2003.823529  
                              5       2004.000000  
                              6       2010.777778  
Transit                       1       2010.282927  
                              2       2012.375000  
                              3       2011.870968  
                              4       2013.272727  
                              5       2012.080000  
                              6       2010.000000  
                              7       2013.000000  
Transit Timing Variations     2       2012.000000  
                              3       2014.000000

In [126]:
def standardize(x):
    z_score = (x - np.mean(x))/np.std(x)
    return z_score
    
planets.groupby(['method'])['orbital_period'].transform(standardize)

0      -0.381158
1       0.035374
2      -0.041521
3      -0.342131
4      -0.211291
5      -0.438573
6       0.653576
7      -0.017099
8       0.116913
9      -0.254920
10      0.041033
11     -0.335891
12     -0.236827
13      0.175181
14      1.078450
15      9.066149
16     -0.563510
17     -0.556342
18     -0.535890
19      2.810689
20     -0.565914
21     -0.386730
22     -0.563521
23     -0.540264
24     -0.481797
25     -0.486146
26     -0.090433
27      0.088982
28     -0.441628
29           NaN
          ...   
1005   -0.377397
1006   -0.360661
1007   -0.357377
1008   -0.395925
1009   -0.348704
1010   -0.285783
1011   -0.364143
1012   -0.373879
1013   -0.361824
1014   -0.362558
1015   -0.423364
1016   -0.407362
1017   -0.368890
1018   -0.357435
1019   -0.394522
1020   -0.409414
1021   -0.403616
1022   -0.427987
1023   -0.410315
1024   -0.378074
1025   -0.390963
1026   -0.437406
1027         NaN
1028   -0.384802
1029         NaN
1030   -0.372023
1031   -0.400762
1032   -0.3882

In [107]:
planets['orbital_period_z'] = planets.groupby(['method'])['orbital_period'].transform(standardize)
planets.head()

,method,number,orbital_period,mass,distance,year,orbital_period_z
0,Radial Velocity,1,269.300,7.10,77.40,2006,-0.381158
1,Radial Velocity,1,874.774,2.21,56.95,2008,0.035374
2,Radial Velocity,1,763.000,2.60,19.84,2011,-0.041521
3,Radial Velocity,1,326.030,19.40,110.62,2007,-0.342131
4,Radial Velocity,1,516.220,10.50,119.47,2009,-0.211291


In [108]:
planets.groupby('method')['orbital_period_z'].mean()

method
Astrometry                      -1.665335e-16
Eclipse Timing Variations       -1.973730e-16
Imaging                         -3.700743e-17
Microlensing                     1.268826e-16
Orbital Brightness Modulation   -1.480297e-16
Pulsar Timing                   -2.220446e-17
Pulsation Timing Variations               NaN
Radial Velocity                  1.039956e-16
Transit                         -8.249768e-18
Transit Timing Variations        0.000000e+00
Name: orbital_period_z, dtype: float64

In [109]:
planets.groupby('method')['orbital_period_z'].std()

method
Astrometry                       1.414214
Eclipse Timing Variations        1.060660
Imaging                          1.044466
Microlensing                     1.080123
Orbital Brightness Modulation    1.224745
Pulsar Timing                    1.118034
Pulsation Timing Variations           NaN
Radial Velocity                  1.000905
Transit                          1.001262
Transit Timing Variations        1.224745
Name: orbital_period_z, dtype: float64

In [133]:
def fano(x):
    fano = np.std(x)/np.mean(x)
    return fano

planets.groupby(['method'])['orbital_period'].transform(fano)

0       1.765473
1       1.765473
2       1.765473
3       1.765473
4       1.765473
5       1.765473
6       1.765473
7       1.765473
8       1.765473
9       1.765473
10      1.765473
11      1.765473
12      1.765473
13      1.765473
14      1.765473
15      1.765473
16      1.765473
17      1.765473
18      1.765473
19      1.765473
20      1.765473
21      1.765473
22      1.765473
23      1.765473
24      1.765473
25      1.765473
26      1.765473
27      1.765473
28      1.765473
29      1.732536
          ...   
1005    2.185932
1006    2.185932
1007    2.185932
1008    2.185932
1009    2.185932
1010    2.185932
1011    2.185932
1012    2.185932
1013    2.185932
1014    2.185932
1015    2.185932
1016    2.185932
1017    2.185932
1018    2.185932
1019    2.185932
1020    2.185932
1021    2.185932
1022    2.185932
1023    2.185932
1024    2.185932
1025    2.185932
1026    2.185932
1027    1.732536
1028    2.185932
1029    1.732536
1030    2.185932
1031    2.185932
1032    2.1859

In [135]:
planets.groupby(['method'])['orbital_period'].aggregate(fano)

method
Astrometry                       0.609683
Eclipse Timing Variations        0.495871
Imaging                          1.732536
Microlensing                     0.326801
Orbital Brightness Modulation    0.835129
Pulsar Timing                    1.987061
Pulsation Timing Variations      0.000000
Radial Velocity                  1.765473
Transit                          2.185932
Transit Timing Variations        0.732746
Name: orbital_period, dtype: float64

In [173]:
def mean_std_fano(x):
    m = np.mean(x) 
    s = np.std(x)
    f = s/m
    n = len(x)
    return (m,s,f,n)

planets.groupby('method')['orbital_period'].aggregate(mean_std_fano)

method
Astrometry                                     (631.18, 384.82, 0.609683450046, 2)
Eclipse Timing Variations        (4751.64444444, 2356.20325069, 0.495871119618, 9)
Imaging                            (118247.7375, 204868.507393, 1.73253638272, 38)
Microlensing                     (3153.57142857, 1030.59176581, 0.326801465942,...
Orbital Brightness Modulation    (0.709306583333, 0.59236215856, 0.835128521966...
Pulsar Timing                     (7343.02120126, 14591.0283028, 1.98706062572, 5)
Pulsation Timing Variations                                  (1170.0, 0.0, 0.0, 1)
Radial Velocity                  (823.354680017, 1453.61012957, 1.76547260233, ...
Transit                           (21.1020726713, 46.12768734, 2.18593159348, 397)
Transit Timing Variations              (79.7835, 58.4610604434, 0.732746250082, 4)
Name: orbital_period, dtype: object

In [174]:
def do_something(x):
    return (x['orbital_period']/x['mass']).mean()

planets.groupby('method').aggregate(do_something)

,number,orbital_period,mass,distance,year,orbital_period_z,orbital_period_z_wrong
method,,,,,,,
Astrometry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Eclipse Timing Variations,1361.711433,1361.711433,1361.711433,1361.711433,1361.711433,1361.711433,1361.711433
Imaging,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Microlensing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Orbital Brightness Modulation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pulsar Timing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pulsation Timing Variations,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Radial Velocity,942.821971,942.821971,942.821971,942.821971,942.821971,942.821971,942.821971
Transit,1.839041,1.839041,1.839041,1.839041,1.839041,1.839041,1.839041


In [175]:
planets.groupby('method').apply(do_something)

method
Astrometry                               NaN
Eclipse Timing Variations        1361.711433
Imaging                                  NaN
Microlensing                             NaN
Orbital Brightness Modulation            NaN
Pulsar Timing                            NaN
Pulsation Timing Variations              NaN
Radial Velocity                   942.821971
Transit                             1.839041
Transit Timing Variations                NaN
dtype: float64

# Even more ways to split

In [177]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data
0   A     0
1   B     1
2   C     2
3   A     3
4   B     4
5   C     5

df.groupby(L).sum()
   data
0     7
1     4
2     4

In [178]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data
key,
A,0
B,1
C,2
A,3
B,4
C,5
,data
consonant,12
vowel,3


In [179]:
display('df2', 'df2.groupby(str.lower).mean()')

,data
key,
A,0
B,1
C,2
A,3
B,4
C,5
,data
a,1.5
b,2.5


In [184]:
decade = 10 * (planets['year'] // 10)
decade.head()

0    2000
1    2000
2    2010
3    2000
4    2000
Name: year, dtype: int64

In [185]:
decade = decade.astype(str) + 's'
decade.head()

0    2000s
1    2000s
2    2010s
3    2000s
4    2000s
Name: year, dtype: object

In [186]:
decade.name = 'decade'

In [187]:
planets.groupby(['method', decade])['number'].sum()

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

In [188]:
planets.groupby(['method', decade])['number'].sum().unstack()

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,NaN,NaN,NaN,2.0
Eclipse Timing Variations,NaN,NaN,5.0,10.0
Imaging,NaN,NaN,29.0,21.0
Microlensing,NaN,NaN,12.0,15.0
Orbital Brightness Modulation,NaN,NaN,NaN,5.0
Pulsar Timing,NaN,9.0,1.0,1.0
Pulsation Timing Variations,NaN,NaN,1.0,NaN
Radial Velocity,1.0,52.0,475.0,424.0
Transit,NaN,NaN,64.0,712.0


In [189]:
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


# Where to go from here